In [1]:
!pip install -q transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# 모델 불러오기
model_name = "kingkim/kodialogpt_v1.1_SecurityManual"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/64.6k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/891k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/496k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [3]:
dataset = load_dataset("kingkim/DS_Building_SecurityManual_V4")

README.md:   0%|          | 0.00/302 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/116 [00:00<?, ? examples/s]

In [8]:
from datasets import load_dataset
from transformers import AutoTokenizer

# 데이터셋 로드
dataset = load_dataset("kingkim/DS_Building_SecurityManual_V4")

# 토크나이저 로드 (모델에 맞게 설정)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 프롬프트 생성 함수
def generate_prompt(data_point):
    input_text = data_point['input']
    output_text = data_point['output']
    return f"Q: {input_text} \nA: {output_text}"

# 전처리 함수
def preprocess_function(examples):
    prompts = [generate_prompt({'input': examples['input'][i], 'output': examples['output'][i]}) for i in range(len(examples['input']))] # Access the elements in the dictionary using keys and then iterate over the lists.
    # Add a padding token to the tokenizer
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    return tokenizer(prompts, truncation=True, padding="max_length", max_length=512)

# 전처리 적용
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.shuffle(seed=42)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/116 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# 모델 로드
model = AutoModelForCausalLM.from_pretrained("kingkim/kodialogpt_v1.1_SecurityManual")

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_dir='./logs',
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True
)

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# 데이터셋 분리
train_test_split = tokenized_dataset['train'].train_test_split(test_size=0.2)
train_data = train_test_split['train']
eval_data = train_test_split['test']

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,11.031400,6.716095
2,4.634600,4.264993


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=33, training_loss=6.970236691561612, metrics={'train_runtime': 79.3066, 'train_samples_per_second': 3.48, 'train_steps_per_second': 0.416, 'total_flos': 68981096448000.0, 'train_loss': 6.970236691561612, 'epoch': 2.869565217391304})

In [13]:
# 모델과 토크나이저 저장
model_directory = "./finetuned_kodialogpt_security_manual"
model.save_pretrained(model_directory)
tokenizer.save_pretrained(model_directory)

# Hugging Face에 업로드
from huggingface_hub import notebook_login
notebook_login()

In [14]:
# Hugging Face에 업로드
model.push_to_hub("kingkim/kodialogpt_v2.0_SecurityManual", use_auth_token=True)
tokenizer.push_to_hub("kingkim/kodialogpt_v2.0_SecurityManual", use_auth_token=True)
trainer.push_to_hub("kingkim/kodialogpt_v2.0_SecurityManual")

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/kingkim/results/commit/0d9fe4e724c8f58d4f9baa03ddacd1c484157e0f', commit_message='kingkim/kodialogpt_v2.0_SecurityManual', commit_description='', oid='0d9fe4e724c8f58d4f9baa03ddacd1c484157e0f', pr_url=None, pr_revision=None, pr_num=None)